# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,2.81,100,75,3.13,AR,1699699137
1,1,puerto ayora,-0.7393,-90.3518,23.64,89,80,1.79,EC,1699699138
2,2,port-aux-francais,-49.3500,70.2167,3.01,66,51,7.92,TF,1699699138
3,3,ust-nera,64.5667,143.2000,-32.43,98,73,1.95,RU,1699699139
4,4,bilibino,68.0546,166.4372,-29.99,100,92,1.56,RU,1699699139


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot


mapplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    title="Humidity",
    size = "Humidity",
    geo = True,
    width = 1400,
    height = 800,
    tiles = "OSM",
    scale = 1.3,
    color="City"
)

# Display the map
#hvplot.show(mapplot)
mapplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_df = city_data_df[(city_data_df["Humidity"] <= 70) & (city_data_df["Max Temp"] <24) & (city_data_df["Max Temp"] >17)]

# Drop any rows with null values
# YOUR CODE HERE

ideal_df = ideal_df.dropna(how='any')

# Display sample data
# YOUR CODE HERE

ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,bredasdorp,-34.5322,20.0403,23.89,40,63,8.31,ZA,1699699152
62,62,port elizabeth,-33.9180,25.5701,23.16,69,20,9.26,ZA,1699698914
70,70,sariveliler,36.6913,32.6196,17.70,34,35,3.91,TR,1699699184
77,77,nova sintra,14.8667,-24.7167,22.23,69,31,2.57,CV,1699699189
78,78,taltal,-25.4000,-70.4833,18.62,69,17,1.39,CL,1699699189


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City","Lat","Lng","Humidity","Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE

hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
20,bredasdorp,-34.5322,20.0403,40,ZA,
62,port elizabeth,-33.9180,25.5701,69,ZA,
70,sariveliler,36.6913,32.6196,34,TR,
77,nova sintra,14.8667,-24.7167,69,CV,
78,taltal,-25.4000,-70.4833,69,CL,
121,hermanus,-34.4187,19.2345,68,ZA,
141,east london,-33.0153,27.9116,68,ZA,
153,amasra,41.7463,32.3863,48,TR,
156,sebastopol,44.6000,33.5333,55,UA,
160,carnarvon,-24.8667,113.6333,70,AU,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
port elizabeth - nearest hotel: Waterford Hotel
sariveliler - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
taltal - nearest hotel: Hostería Taltal
hermanus - nearest hotel: Aloe guest house
east london - nearest hotel: No hotel found
amasra - nearest hotel: Liman Hotel
sebastopol - nearest hotel: Барабулька
carnarvon - nearest hotel: No hotel found
piranshahr - nearest hotel: هتل چهار ستاره زاگرس
myitkyina - nearest hotel: New Light Hotel
arhribs - nearest hotel: No hotel found
berriane - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
maralal - nearest hotel: Peacock Rest House
metlili chaamba - nearest hotel: No hotel found
tadmur - nearest hotel: فندق فيلا بالميرا
namibe - nearest hotel: Hotel Chik Chik Namibe
hazro city - nearest hotel: No hotel found
benalla - nearest hotel: Hotel Haven
narrabri - nearest hotel: Tattersall's Hotel
constantia - nearest hote

,City,Lat,Lng,Humidity,Country,Hotel Name
20,bredasdorp,-34.5322,20.0403,40,ZA,Victoria Hotel
62,port elizabeth,-33.9180,25.5701,69,ZA,Waterford Hotel
70,sariveliler,36.6913,32.6196,34,TR,No hotel found
77,nova sintra,14.8667,-24.7167,69,CV,Residência Ka Dencho
78,taltal,-25.4000,-70.4833,69,CL,Hostería Taltal
121,hermanus,-34.4187,19.2345,68,ZA,Aloe guest house
141,east london,-33.0153,27.9116,68,ZA,No hotel found
153,amasra,41.7463,32.3863,48,TR,Liman Hotel
156,sebastopol,44.6000,33.5333,55,UA,Барабулька
160,carnarvon,-24.8667,113.6333,70,AU,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    hover_cols=["Hotel Name","Country"],
    scale = 1,
    title="Ideal Conditions",
    color="City"
    
)

# Display the map
# YOUR CODE HERE
#hvplot.show(hotel_map)
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)